In [3]:
import numpy as np
from mindquantum.core.gates import RX, RY, RZ, H, X, Y, Z, CNOT
from mindquantum.core.circuit import Circuit
import mindspore as ms
from mindquantum.simulator import  Simulator
from mindquantum.core.gates import GroupedPauli
from mindquantum.core.operators import TimeEvolution,QubitOperator
from mindquantum.core.parameterresolver import PRConvertible,PRGenerator,ParameterResolver
from DQAS_tool import generate_pauli_string,one_hot
from mindquantum.core.gates import RotPauliString
from mindquantum.core.gates import UnivMathGate
from mindspore import Tensor, ops
from mindquantum.core.circuit import UN
from mindquantum.core.operators import Hamiltonian             # 导入Hamiltonian模块，用于构建哈密顿量
from mindquantum.framework import MQLayer
from mindspore.nn import  TrainOneStepCell
from mindspore.nn import SoftmaxCrossEntropyWithLogits                         # 导入SoftmaxCrossEntropyWithLogits模块，用于定义损失函数
from mindspore.nn import Adam                                                  # 导入Adam模块用于定义优化参数
from mindspore.train import Accuracy, Model, LossMonitor                       # 导入Accuracy模块，用于评估预测准确率
import mindspore as ms
from mindspore import Parameter, Tensor
from mindspore.dataset import NumpySlicesDataset
from torch.utils.data import DataLoader# 导入NumpySlicesDataset模块，用于创建模型可以识别的数据集
import sys
sys.path.append('..')
from data_processing import X_train,X_test,y_train,y_test
from mindquantum.core.operators import TimeEvolution,QubitOperator
from mindquantum.core.parameterresolver import PRGenerator
num_layer = 3
# 定义标准差和形状
stddev = 0.02
shape_nnp = (num_layer, 8)
shape_stp = (num_layer, 12)

rtype = np.float64
ctype = np.complex128
# 使用 numpy 生成随机数矩阵
nnp = np.random.normal(loc=0.0, scale=stddev, size=shape_nnp).astype(rtype)
stp = np.random.normal(loc=0.0, scale=stddev, size=shape_stp).astype(rtype)
#Operator Pool
unbound_opeartor_pool = [generate_pauli_string(n=8,seed=i)[0] for i in range(8)]
bound_opeartor_pool = [generate_pauli_string(n=8,seed=i)[1] for i in range(8,12)]

/opt/miniconda3/envs/MindSpore/lib/python3.9/site-packages/mindquantum/simulator/__init__.py:17: UserWarning: Unable import mqvector gpu backend due to: cannot import name '_mq_vector_gpu' from partially initialized module 'mindquantum' (most likely due to a circular import) (/opt/miniconda3/envs/MindSpore/lib/python3.9/site-packages/mindquantum/__init__.py)
  from .available_simulator import SUPPORTED_SIMULATOR


In [17]:
ansatz_pr_gen = PRGenerator(name='Pool')
parameterized_pool = \
[TimeEvolution(QubitOperator(terms='I0 I1 X2 X3',coefficient=ansatz_pr_gen.new()),time=0.5)]

In [18]:
TimeEvolution(QubitOperator(terms='I0 I1 X2 X3',coefficient=ansatz_pr_gen.new()),time=0.5).circuit

┏━━━┓                           ┏━━━┓   
q2: ──┨ H ┠───■───────────────────■───┨ H ┠───
      ┗━━━┛   ┃                   ┃   ┗━━━┛   
      ┏━━━┓ ┏━┻━┓ ┏━━━━━━━━━━━┓ ┏━┻━┓ ┏━━━┓   
q3: ──┨ H ┠─┨╺╋╸┠─┨ RZ(Pool1) ┠─┨╺╋╸┠─┨ H ┠───
      ┗━━━┛ ┗━━━┛ ┗━━━━━━━━━━━┛ ┗━━━┛ ┗━━━┛

In [33]:
from mindquantum.core.circuit import UN
from mindquantum.core.gates import X,RX
circuit1 = UN(RX, maps_obj = [0,1], maps_ctrl = [2, 3])
print(circuit1)

TypeError: Require a quantum gate, but get <class 'abc.ABCMeta'>

In [34]:
from mindquantum.core.gates import SWAP
circuit2 = UN(SWAP, maps_obj =[[0, 1], [2, 3]]).x(2, 1)
print(circuit2)

q0: ──╳─────────
      ┃         
      ┃         
q1: ──╳───■─────
          ┃     
        ┏━┻━┓   
q2: ──╳─┨╺╋╸┠───
      ┃ ┗━━━┛   
      ┃         
q3: ──╳─────────


In [10]:
net = Mindspore_ansatz(Structure_p=stp,Ansatz_p=nnp,n_layer=3,n_qbits=8)

In [ ]:
print(net.trainable_params())


In [20]:
loss_fn = ms.nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean') # 定义损失函数    
def forward_fn(x, y):
    z = net(x)
    print(z)
    loss = loss_fn(z, y)
    return loss

In [ ]:
grad_fn = ms.value_and_grad(forward_fn, None, weights=net.trainable_params())
loss, grads = grad_fn(X_train[0],y_train[0])
print(grads)


In [ ]:
g

In [ ]:


def fn(x, y, z):
    res = x * ops.exp(y) * ops.pow(z, 2)
    return res, z

x = Tensor([3, 3], ms.float32)
y = Tensor([0, 0], ms.float32)
z = Tensor([5, 5], ms.float32)
gradient, aux = grad(fn, (1, 2), None, True,True)(x, y, z)
gradient

In [ ]:
aux

In [ ]:
nnp = np.random.normal(loc=0.0, scale=stddev, size=shape_nnp).astype(rtype)
stp = np.random.normal(loc=0.0, scale=stddev, size=shape_stp).astype(rtype)

def wrapped_mindspore_ansatz(Structure_p, Ansatz_p, n_layer=3, n_qbits=8):
    return Mindspore_ansatz(Structure_p, Ansatz_p, n_layer=n_layer, n_qbits=n_qbits)

# 创建带梯度计算的函数
vag2 = ops.value_and_grad(fn=wrapped_mindspore_ansatz, grad_position=(0, 1))

# 使用 vag2 计算损失及其对 Structure_p 和 Ansatz_p 的梯度
loss_value, (grad_structure, grad_ansatz) = vag2(stp, nnp)  # 确保 stp 和 nnp 是 numpy array
print("Loss Value:", loss_value)
print("Gradient for Structure_p:", grad_structure)
print("Gradient for Ansatz_p:", grad_ansatz)



In [ ]:
Mindspore_ansatz(stp,nnp,3,8)

In [ ]:
vag2(stp,nnp,3,8)